In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

In [7]:
!pip install seaborn xgboost optuna

In [8]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
df_submission = pd.read_csv("sample_submission.csv")

In [18]:
df_test.head()

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment
0,750000,Educational Nuggets,Episode 73,78.96,Education,38.11,Saturday,Evening,53.33,1.0,Neutral
1,750001,Sound Waves,Episode 23,27.87,Music,71.29,Sunday,Morning,NaN,0.0,Neutral
2,750002,Joke Junction,Episode 11,69.10,Comedy,67.89,Friday,Evening,97.51,0.0,Positive
3,750003,Comedy Corner,Episode 73,115.39,Comedy,23.40,Sunday,Morning,51.75,2.0,Positive
4,750004,Life Lessons,Episode 50,72.32,Lifestyle,58.10,Wednesday,Morning,11.30,2.0,Neutral


In [20]:
df_test.head()

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment
0,750000,Educational Nuggets,Episode 73,78.96,Education,38.11,Saturday,Evening,53.33,1.0,Neutral
1,750001,Sound Waves,Episode 23,27.87,Music,71.29,Sunday,Morning,NaN,0.0,Neutral
2,750002,Joke Junction,Episode 11,69.10,Comedy,67.89,Friday,Evening,97.51,0.0,Positive
3,750003,Comedy Corner,Episode 73,115.39,Comedy,23.40,Sunday,Morning,51.75,2.0,Positive
4,750004,Life Lessons,Episode 50,72.32,Lifestyle,58.10,Wednesday,Morning,11.30,2.0,Neutral


In [14]:
df_train.describe()

,id,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes
count,750000.000000,662907.000000,750000.000000,603970.000000,749999.000000,750000.000000
mean,374999.500000,64.504738,59.859901,52.236449,1.348855,45.437406
std,216506.495284,32.969603,22.873098,28.451241,1.151130,27.138306
min,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000
25%,187499.750000,35.730000,39.410000,28.380000,0.000000,23.178350
50%,374999.500000,63.840000,60.050000,53.580000,1.000000,43.379460
75%,562499.250000,94.070000,79.530000,76.600000,2.000000,64.811580
max,749999.000000,325.240000,119.460000,119.910000,103.910000,119.970000


In [12]:
df_train.isnull().sum()

id                                  0
Podcast_Name                        0
Episode_Title                       0
Episode_Length_minutes          87093
Genre                               0
Host_Popularity_percentage          0
Publication_Day                     0
Publication_Time                    0
Guest_Popularity_percentage    146030
Number_of_Ads                       1
Episode_Sentiment                   0
Listening_Time_minutes              0
dtype: int64

In [22]:
df_test = df_test.set_index('id', inplace=False)

In [23]:
df_train = df_train.set_index('id', inplace=False)

In [27]:
X = df_train.drop(columns = [ 'Listening_Time_minutes'], axis = 1)
y = df_train["Listening_Time_minutes"]
#df_test = df_test.drop(columns = ['id'], axis =1)

In [28]:
df_test.head()

,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment
id,,,,,,,,,,
750000,Educational Nuggets,Episode 73,78.96,Education,38.11,Saturday,Evening,53.33,1.0,Neutral
750001,Sound Waves,Episode 23,27.87,Music,71.29,Sunday,Morning,NaN,0.0,Neutral
750002,Joke Junction,Episode 11,69.10,Comedy,67.89,Friday,Evening,97.51,0.0,Positive
750003,Comedy Corner,Episode 73,115.39,Comedy,23.40,Sunday,Morning,51.75,2.0,Positive
750004,Life Lessons,Episode 50,72.32,Lifestyle,58.10,Wednesday,Morning,11.30,2.0,Neutral


In [29]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
label_encoder = LabelEncoder()

# Apply label encoding to each categorical column
categorical_columns = ['Podcast_Name', 'Episode_Title', 'Genre', 'Publication_Day', 'Publication_Time', 'Episode_Sentiment']
for col in categorical_columns:
    X[col] = label_encoder.fit_transform(X[col])
    
    df_test[col] = label_encoder.transform(df_test[col])

In [30]:
df_test.head()

,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment
id,,,,,,,,,,
750000,11,71,78.96,2,38.11,2,1,53.33,1.0,1
750001,36,16,27.87,5,71.29,3,2,NaN,0.0,1
750002,24,3,69.10,1,67.89,0,1,97.51,0.0,2
750003,4,71,115.39,1,23.40,3,2,51.75,2.0,2
750004,27,46,72.32,4,58.10,6,2,11.30,2.0,1


In [31]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test= train_test_split(X,y, test_size= 0.2, random_state=42)

In [75]:
X_train.shape

(600000, 10)

## Model XGBoost

In [40]:
import xgboost as xgb
from xgboost import XGBRegressor

model_xgb = XGBRegressor(**params)
model_xgb.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=100)

[0]	validation_0-rmse:27.06098	validation_1-rmse:27.04630
[100]	validation_0-rmse:20.87231	validation_1-rmse:20.92039
[200]	validation_0-rmse:16.21559	validation_1-rmse:16.32550
[300]	validation_0-rmse:13.69412	validation_1-rmse:13.88752
[400]	validation_0-rmse:12.85158	validation_1-rmse:13.13576
[500]	validation_0-rmse:12.62692	validation_1-rmse:12.99588
[600]	validation_0-rmse:12.50828	validation_1-rmse:12.95276


[700]	validation_0-rmse:12.42475	validation_1-rmse:12.93322
[800]	validation_0-rmse:12.34423	validation_1-rmse:12.91768
[900]	validation_0-rmse:12.23440	validation_1-rmse:12.89778
[1000]	validation_0-rmse:12.13147	validation_1-rmse:12.88168
[1100]	validation_0-rmse:12.02499	validation_1-rmse:12.86645
[1200]	validation_0-rmse:11.92743	validation_1-rmse:12.85268
[1300]	validation_0-rmse:11.83379	validation_1-rmse:12.84075
[1400]	validation_0-rmse:11.74219	validation_1-rmse:12.83040
[1500]	validation_0-rmse:11.64896	validation_1-rmse:12.81974
[1600]	validation_0-rmse:11.56203	validation_1-rmse:12.81091
[1700]	validation_0-rmse:11.47940	validation_1-rmse:12.80329
[1800]	validation_0-rmse:11.39181	validation_1-rmse:12.79511
[1900]	validation_0-rmse:11.30905	validation_1-rmse:12.78695
[2000]	validation_0-rmse:11.23066	validation_1-rmse:12.78013
[2100]	validation_0-rmse:11.14610	validation_1-rmse:12.77335
[2200]	validation_0-rmse:11.06832	validation_1-rmse:12.76703
[2300]	validation_0-rmse:10

XGBRegressor(alpha=0.010101632821320981, base_score=None, booster='gbtree',
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.9367557190770411, device='cuda',
             early_stopping_rounds=100, enable_categorical=False,
             eta=0.012528084926241448, eval_metric='rmse', feature_types=None,
             feature_weights=None, gamma=7.340223431497053, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             lambda=0.04186421414163593, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None, max_delta_step=9,
             max_depth=10, max_leaves=None, min_child_weight=3, missing=nan,
             monotone_constraints=None, multi_strategy=None, ...)

In [41]:
#df_test.to_device("cuda")
df_submission['Listening_Time_minutes'] = model_xgb.predict(df_test)

In [42]:
df_submission.head()

,id,Listening_Time_minutes
0,750000,53.691082
1,750001,19.662548
2,750002,50.899666
3,750003,72.352051
4,750004,46.127178


In [44]:
df_submission.to_csv('submission12.65.csv', index=False)

## Optuna

In [39]:
params = {'lambda': 0.04186421414163593,
 'alpha': 0.010101632821320981,
 'subsample': 0.9737046239306681,
 'colsample_bytree': 0.9367557190770411,
 'max_depth': 10,
 'eta': 0.012528084926241448,
 'gamma': 7.340223431497053,
 'scale_pos_weight': 2.237856577831878,
 'min_child_weight': 3,
 'max_delta_step': 9,
 'objective': 'reg:squarederror',
 'eval_metric': 'rmse',
 'verbosity': 0,
 'booster': 'gbtree',
 'device' : 'cuda',
 'early_stopping_rounds':100,
 'n_estimators' : 18000}

In [34]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

def objective(trial):
    param = {
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'verbosity': 0,
        'booster': 'gbtree',
        'lambda': trial.suggest_float('lambda', 1e-3, 10.0, log=True),
        'alpha': trial.suggest_float('alpha', 1e-3, 10.0, log=True),
        'subsample': trial.suggest_float('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.4, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'eta': trial.suggest_float('eta', 0.01, 0.3),
        'gamma': trial.suggest_float('gamma', 0, 10),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 1, 10),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10),
        'max_delta_step': trial.suggest_int('max_delta_step', 0, 10),
        'device' : 'cuda',
        'early_stopping_rounds':100,
        'n_estimators' : 18000
        
    }

    model = xgb.XGBRegressor(**param)

    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=0)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Calculate the root mean squared error
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    return rmse


# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

# Print the best hyperparameters and the corresponding RMSE
print("Best hyperparameters:", study.best_params)
print("Best RMSE:", study.best_value)

[I 2025-04-11 16:57:26,699] A new study created in memory with name: no-name-e4048fb3-4fca-45dc-bfb3-83e6147d9363
[I 2025-04-11 16:57:29,810] Trial 0 finished with value: 12.937489763108148 and parameters: {'lambda': 0.007056099601144256, 'alpha': 0.08406347159516234, 'subsample': 0.4808635421963215, 'colsample_bytree': 0.8396790506390786, 'max_depth': 7, 'eta': 0.13190983438507395, 'gamma': 1.8894358404991929, 'scale_pos_weight': 3.404872410646255, 'min_child_weight': 5, 'max_delta_step': 4}. Best is trial 0 with value: 12.937489763108148.
[I 2025-04-11 16:57:30,878] Trial 1 finished with value: 12.996185893173548 and parameters: {'lambda': 1.3430351727267744, 'alpha': 0.020296823474505, 'subsample': 0.7297398976347488, 'colsample_bytree': 0.609273501703208, 'max_depth': 7, 'eta': 0.28248733179225727, 'gamma': 2.714315930668143, 'scale_pos_weight': 8.649424826202123, 'min_child_weight': 1, 'max_delta_step': 7}. Best is trial 0 with value: 12.937489763108148.
[I 2025-04-11 16:57:32,717

Best hyperparameters: {'lambda': 0.04186421414163593, 'alpha': 0.010101632821320981, 'subsample': 0.9737046239306681, 'colsample_bytree': 0.9367557190770411, 'max_depth': 10, 'eta': 0.012528084926241448, 'gamma': 7.340223431497053, 'scale_pos_weight': 2.237856577831878, 'min_child_weight': 3, 'max_delta_step': 9}
Best RMSE: 12.650002826702824


In [36]:
Best_hyperparameters = study.best_params

In [37]:
Best_hyperparameters

{'lambda': 0.04186421414163593,
 'alpha': 0.010101632821320981,
 'subsample': 0.9737046239306681,
 'colsample_bytree': 0.9367557190770411,
 'max_depth': 10,
 'eta': 0.012528084926241448,
 'gamma': 7.340223431497053,
 'scale_pos_weight': 2.237856577831878,
 'min_child_weight': 3,
 'max_delta_step': 9}

In [87]:
import xgboost as xgb
from xgboost import XGBRegressor

model_xgb = XGBRegressor(**study.best_params)
model_xgb.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_test, y_test)], verbose=100)

[0]	validation_0-rmse:26.77346	validation_1-rmse:26.75831
[99]	validation_0-rmse:13.12113	validation_1-rmse:13.11336


XGBRegressor(alpha=0.025069364993560855, base_score=None, booster=None,
             callbacks=None, colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.964036641454417, device=None,
             early_stopping_rounds=None, enable_categorical=False,
             eta=0.07393409653944201, eval_metric=None, feature_types=None,
             gamma=9.222692117951766, grow_policy=None, importance_type=None,
             interaction_constraints=None, lambda=0.0016865621189580064,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=7, max_depth=6,
             max_leaves=None, min_child_weight=6, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None, ...)

In [81]:


def objective(trial):
    """Objective function for Optuna optimization."""

    # Define the hyperparameter search space
    xgb_learning_rate = trial.suggest_float("xgb_learning_rate", 0.01, 0.1)
    xgb_max_depth = trial.suggest_int("xgb_max_depth", 3, 10)
    xgb_n_estimators = trial.suggest_int("xgb_n_estimators", 1000, 20000, step=500)
    #xgb_subsample = trial.suggest_float("xgb_subsample", 0.5, 1.0)
    #xgb_colsample_bytree = trial.suggest_float("xgb_colsample_bytree", 0.5, 1.0)

    # Define the XGBoost regressor with the suggested hyperparameters
    model_xgb = xgb.XGBRegressor(
        objective='reg:squarederror',
        n_estimators=xgb_n_estimators,
        learning_rate=xgb_learning_rate,
        max_depth=xgb_max_depth,
        random_state=42,
        tree_method='hist',  # Use hist for faster training
        device = "cuda",
        #subsample=xgb_subsample,
        #colsample_bytree=xgb_colsample_bytree,
        early_stopping_rounds=100
    )

    # Train the model and evaluate on the validation set
    model_xgb.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False)

    # Make predictions on the test set
    y_pred = model_xgb.predict(X_test)

    # Calculate the root mean squared error
    rmse = mean_squared_error(y_test, y_pred, squared=False)

    return rmse


# Create an Optuna study and optimize the objective function
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=10)

# Print the best hyperparameters and the corresponding RMSE
print("Best hyperparameters:", study.best_params)
print("Best RMSE:", study.best_value)

# Train the final model with the best hyperparameters
best_model_xgb = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=study.best_params['xgb_n_estimators'],
    learning_rate=study.best_params['xgb_learning_rate'],
    max_depth=study.best_params['xgb_max_depth'],
    random_state=42,
    tree_method='hist',  # Use hist for faster training
    device = "cuda",
    #subsample=study.best_params['xgb_subsample'],
    #colsample_bytree=study.best_params['xgb_colsample_bytree'],
    early_stopping_rounds=None  # Disable early stopping for final training
)

best_model_xgb.fit(X_train, y_train)

[I 2025-04-11 16:23:10,080] A new study created in memory with name: no-name-f57270da-bd15-4813-a5df-a6610ecd0558
[I 2025-04-11 16:23:28,487] Trial 0 finished with value: 12.937814567323842 and parameters: {'xgb_learning_rate': 0.08152816565383289, 'xgb_max_depth': 4, 'xgb_n_estimators': 3500}. Best is trial 0 with value: 12.937814567323842.
[I 2025-04-11 16:23:45,395] Trial 1 finished with value: 12.834809187498587 and parameters: {'xgb_learning_rate': 0.06990807893172166, 'xgb_max_depth': 7, 'xgb_n_estimators': 12500}. Best is trial 1 with value: 12.834809187498587.
[I 2025-04-11 16:24:18,666] Trial 2 finished with value: 12.864219722416639 and parameters: {'xgb_learning_rate': 0.039388680841069015, 'xgb_max_depth': 6, 'xgb_n_estimators': 14000}. Best is trial 1 with value: 12.834809187498587.
[I 2025-04-11 16:24:34,467] Trial 3 finished with value: 12.868499266311805 and parameters: {'xgb_learning_rate': 0.08807568193797817, 'xgb_max_depth': 6, 'xgb_n_estimators': 11000}. Best is tr

KeyboardInterrupt: 